In [1]:
import os
import sys

sys.path.append("..")  # Adds higher directory to python modules path.
from src import RASPRoutines
import polars as pl
import numpy as np
from src import IOFunctions
from src import AnalysisFunctions

RASP = RASPRoutines.RASP_Routines()
IO = IOFunctions.IO_Functions()
A_F = AnalysisFunctions.Analysis_Functions()

import warnings
from copy import copy

warnings.filterwarnings("ignore")

from src import HelperFunctions
H_F = HelperFunctions.Helper_Functions()


In [ ]:
# MAKE SURE THESE POINT TO THE CORRECT FOLDERS
overall_folder = r"S:\ASAP_Imaging_Data\Main_Survey\20241105_oligomers_in_cells"
raw_data_folder = r"S:\ASAP_Imaging_Data\Main_Survey\Raw_data_ss"

In [ ]:
brainregion_folders = np.sort(os.listdir(overall_folder))

protein_strings = np.array(["C1", "C1", "C1", "C1", "C1", "C1", "C1", "C1", "C1"])
cell_strings = np.array(["C0", "C0", "C0", "C0", "C0", "C0", "C0", "C0", "C0"])

cell_stains = np.array(["microglia", "neurons"])
patient_status = np.array(["PD", "HC"])
imtype = ".tif"
cell_size_threshold = np.array(
    [2000.0, 2000.0, 2000.0, 2000.0]
)  # first one for microglia, second for neurons, third for astrocytes, fourth for oligodendrocytes
percentiles = np.arange(0, 100, 5)

In [ ]:
for q, region in enumerate(brainregion_folders):
    protein_string = protein_strings[q]
    cell_string = cell_strings[q]
    for i, cell_stain in enumerate(cell_stains):
        for j, patient in enumerate(patient_status):
            """the command below will run the analyse_images analysis:

                folder (string): Folder containing images
                imtype (string): Type of images being analysed. Default '.tif'
                thres (float): fraction of bright pixels accepted. Default 0.05.
                large_thres (float): large object intensity threshold. Default 100.
                gisgma (float): gaussian blurring parameter. Default 1.4.
                rwave (float): Ricker wavelent sigma. Default 2.
                protein_string (np.1darray): strings for protein-stained data. Default C1.
                cell_string (np.1darray): strings for cell-containing data. Default C0.
                if_filter (boolean): Filter images for focus. Default True.
                im_start (integer): Images to start from. Default 0.
                cell_analysis (boolean): Parameter where script also analyses cell
                    images and computes colocalisation likelihood ratios. Default True.
                one_savefile (boolean): Parameter that, if true, doesn't save a file. Default True.
                    per image but amalgamates them into one file. Default True.
                disp (boolean): If true, prints when analysed an image stack. Default True.
                analyse_clr (boolean): If true, calculates the clr. If not, just coincidence. Default True.
                error_reduction (boolean): If true, reduces error on the oligomer image using Huang's code

            All have defaults except folder, which is necessary.
            """
        # then, get the percentile brightness from HC
        analysis_directory_HC = os.path.join(
            overall_folder, region, cell_stain, "HC" + "_analysis"
        )
        analysis_directory_PD = os.path.join(
            overall_folder, region, cell_stain, "PD" + "_analysis"
        )
        savename_HC = os.path.join(analysis_directory_HC, "spot_analysis.csv")
        savename_PD = os.path.join(analysis_directory_PD, "spot_analysis.csv")
        if os.path.isfile(savename_HC) and os.path.isfile(savename_PD):
            HC_data = pl.read_csv(savename_HC)
            PD_data = pl.read_csv(savename_PD)
            if "incell" in PD_data:
                PD_data = PD_data.drop("incell")
            if "incell" in HC_data:
                HC_data = HC_data.drop("incell")
            overall_region = pl.concat([HC_data, PD_data], rechunk=True)
            del PD_data
            del HC_data
            overall_region_filtered, q1, q2, IQR = A_F.reject_outliers(
                overall_region, k=4
            )
            del overall_region
            thresholds = np.percentile(
                overall_region_filtered["sum_intensity_in_photons"].to_numpy(),
                percentiles,
            )
            del overall_region_filtered
            if os.path.isfile(savename_HC):
                for patient in patient_status:
                    analysis_directory = os.path.join(
                        overall_folder, region, cell_stain, patient + "_analysis"
                    )
                    analysis_file = os.path.join(
                        analysis_directory, "spot_analysis.csv"
                    )
                    print("Analysing " + str(analysis_file))
                    analysis_data_raw, _, _, _ = A_F.reject_outliers(
                        pl.read_csv(analysis_file), q1=q1, q2=q2, IQR=IQR, k=4
                        )
                    for t, threshold in enumerate(thresholds):
                        percentile = percentiles[t]
                        end_str = "combined_threshold"+"_percentile_"+str(int(percentile))+"_k_4_outliersremoved"
                        cell_punctum_analysis = (
                            RASP.count_puncta_in_individual_cells_threshold_3D(
                                copy(analysis_data_raw),
                                analysis_file,
                                threshold,
                                cell_string,
                                protein_string,
                                lower_cell_size_threshold=cell_size_threshold[i],
                                upper_cell_size_threshold=np.inf,
                                imtype=".tif",
                                blur_degree=1,
                                replace_files=False,
                                end_string=end_str,
                                dims=3,
                                sigma1=2.0,
                                sigma2=40.0,
                                spacing=(0.5, 0.11, 0.11),
                            )
                        )
                        del cell_punctum_analysis